In [1]:
from ortools.sat.python import cp_model
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import transmap
import psycopg2
from pymongo import MongoClient
import math

# COE Dredge Data Processing

In [ ]:
psql_conn = psycopg2.connect(host = "compass.cast.uark.edu",
                        database = "transmap",
                        user = "transmapRead",
                        password = "2ogHwuEbJi7n0UfxAiz5YrtUCcqXzF",
                        port = 5432)
cur = psql_conn.cursor()

In [ ]:
sql = "select * from usace.coe_dredge_locations;"
coe_dredge_data = gpd.read_postgis(sql=sql, con=psql_conn, geom_col='geometry')
dredge_dict = {
    'B': 'Bucket',
    'D': 'Dustpan',
    'H': 'Hopper',
    'I': 'Water Injection',
    'N': 'Non-Conventional',
    'P': 'Pipeline',
    'U': 'Unknown',
    'S': 'Sidecaster',
    'W': 'Combo all types',
    'X': 'Pipeline + Bucket',
    'Y': 'Pipeline + Hopper',
    'Z': 'Hopper + Bucket'
}
coe_dredge_data['start_date'] = coe_dredge_data['STARTDAT']
coe_dredge_data['end_date'] = coe_dredge_data['STOPDAT']
coe_dredge_data['dredge_name'] = coe_dredge_data['PRDRNAME']
coe_dredge_data['job_name'] = coe_dredge_data['JOBNAME']
coe_dredge_data['district_name'] = coe_dredge_data['DISTNAME']
coe_dredge_data['dredge_type'] = coe_dredge_data['PRDRTYPE'].map(dredge_dict)
coe_dredge_data.loc[coe_dredge_data['dredge_type'].isnull(), 'dredge_type'] = 'Unknown'
coe_dredge_data['cubic_yards'] = coe_dredge_data['ACTUALCY']
coe_dredge_data.loc[coe_dredge_data['cubic_yards'].isnull(), 'cubic_yards'] = 0
coe_dredge_data['work_days'] = coe_dredge_data['NUMWKDY']
coe_dredge_data.loc[coe_dredge_data['work_days'].isnull(), 'work_days'] = 0
coe_dredge_data['cost'] = coe_dredge_data['ACTUAL_COS']
coe_dredge_data.loc[coe_dredge_data['cost'].isnull(), 'cost'] = 0
coe_dredge_data['production_rate'] = coe_dredge_data['cubic_yards']/coe_dredge_data['work_days']
coe_dredge_data = coe_dredge_data[['FY', 'start_date', 'end_date', 'job_name', 'district_name', 'dredge_name',
                                   'dredge_type', 'cost', 'cubic_yards', 'work_days', 'production_rate', 'geometry']]
coe_dredge_data = coe_dredge_data.loc[coe_dredge_data['cubic_yards']>0]
coe_dredge_data = coe_dredge_data.loc[coe_dredge_data['dredge_name']!=' ']
coe_dredge_data

In [ ]:
coe_dredge_data.groupby('dredge_name').count()

In [ ]:
for name, dredge in coe_dredge_data.groupby('dredge_name'):
    for district_name, district in dredge.groupby(['district_name', 'dredge_type']):
        print(name, district_name)
        print(dredge['production_rate'].mean())

# Model Creation

In [82]:
model = cp_model.CpModel()

In [83]:
#Sets
#====

with open('32J/ExperimentInfo32j30DE.txt') as f:
    lines = f.readlines()
    J = int(lines[0])
    D = int(lines[1])
    T = int(lines[2])
    W = int(lines[3])
    B = int(lines[4])


In [84]:
#Parameters
#==========
bw = {} #begining of restricted period
ew = {} #end of restricted period
rd = {} #operation rate of dredge d
qj = {} #dredgin amount of job j
tdj = {} #time in days for dredge d to complete job j
tjj = {} #time for dredge d to move to job j to j'
cj = {} #cost for completing job j

for j in range(1, J+1):
    bw[j] = []
    ew[j] = []
    
with open('32J/RP.txt') as f:
    for line in f:
        line_split = line.split('\t')
        j = int(line_split[0])
        bw[j].append(int(line_split[1]))
        ew[j].append(int(line_split[2]))
f.close()

with open('32J/OpRates.txt') as f:
    d = 1
    for line in f:
        if d > D:
            break
        else:
            rd[d] = int(line)
            d += 1
f.close()

with open('32J/JobSizes.txt') as f:
    j = 1
    for line in f:
        qj[j] = int(line)
        j+=1
f.close()

with open('32J/Cost.txt') as f:
    j = 1
    for line in f:
        cj[j] = int(line)
        j+=1
f.close()

for d in range(1, D+1):
    for j in range(1, J+1):
        tdj[d,j] = math.floor(qj[j] / rd[d]) + 1

with open('32J/TravelTimeMatrix.txt') as f:
    j1 = 1
    for line in f:
        line_split = line.split('\t')
        j2 = 1
        for time in line_split:
            tjj[j1,j2] = int(time)
            j2+=1
        j1+=1
        
restricted_period = {}
w = 1
for j in range(1, J+1):
    restricted_period[j] = []
    for b, e in zip(bw[j], ew[j]):
        restricted_period[j].append(model.NewIntervalVar(b, e-b, e, 'restricted_period_{}'.format(w)))
        w+=1

In [85]:
#Decision Variables
#==================
start_var = {}
end_var = {}
is_present = {}
interval_var = {}
precedence = {}
for d in range(1, D+1):
    for j in range(1, J+1):
        start_var[d,j] = model.NewIntVar(0, T, 'start_{}_{}'.format(d,j))
        end_var[d,j] = model.NewIntVar(0, T, 'end_{}_{}'.format(d,j))
        is_present[d,j] = model.NewBoolVar('is_present_{}_{}'.format(d,j))
        interval_var[d,j] = model.NewOptionalIntervalVar(start_var[d,j], tdj[d,j], end_var[d,j], is_present[d,j], 'interval_{}_{}'.format(d,j))
        for j_ in range(1, J+1):
            if j != j_:
                precedence[d,j,j_] = model.NewBoolVar('precedence_{}_{}'.format(d,j))

In [86]:
#Constraints
#===========

#One dredge per job
for j in range(1, J+1):
    model.Add(sum(is_present[d,j] for d in range(1, D+1)) <= 1)

In [87]:
#Budget Constraint
#=================
model.Add(sum(cj[j] * is_present[d,j] for d in range(1, D+1) for j in range(1, J+1)) <= B)

In [88]:
#Dredge cant start another job until current one is finished
#===========================================================
for d in range(1, D+1):
    model.AddNoOverlap([interval_var[d,j] for j in range(1, J+1)])
    
for d in range(1, D+1):
    for j1 in range(1, J+1):
        for j2 in range(1, J+1):
            if j1 < j2:
                model.Add(precedence[d,j1,j2] + precedence[d,j2,j1] == 1).OnlyEnforceIf([is_present[d,j1],is_present[d,j2]])
            if j1 != j2:
                model.Add(start_var[d,j2] >= end_var[d,j1] + tjj[j1,j2]).OnlyEnforceIf(precedence[d,j1,j2])


In [89]:
#Job can't be performed in restricted period
#===========================================

for d in range(1, D+1):
    for j in range(1, J+1):
        for w in restricted_period[j]:
            model.AddNoOverlap([interval_var[d,j], w])
            

In [90]:
#Completion Time cannot exceed planning horizon
#==============================================
for d in range(1, D+1):
    for j in range(1, J+1):
        model.Add( end_var[d,j] <= T )

In [91]:
#Objective
#=========
model.Maximize(sum(qj[j] * is_present[d,j] for d in range(1, D+1) for j in range(1,J+1)))

In [92]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10.0
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model)

In [93]:
solver.ObjectiveValue()

8413704.0

In [95]:
cost = []
for d in range(1, D+1):
    for j in range(1, J+1):
        if solver.Value(is_present[d,j]) == 1:
            print('Dredge {} performing job {} at time {} - {} ({} days)'.format(d,j,solver.Value(start_var[d,j]), solver.Value(end_var[d,j]), tdj[d,j]))
            cost.append(cj[j])
print(sum(cost))

Dredge 8 performing job 3 at time 60 - 62 (2)
Dredge 9 performing job 7 at time 0 - 14 (14)
Dredge 10 performing job 1 at time 0 - 133 (133)
Dredge 11 performing job 10 at time 0 - 2 (2)
Dredge 12 performing job 5 at time 0 - 126 (126)
Dredge 12 performing job 12 at time 151 - 179 (28)
Dredge 13 performing job 13 at time 0 - 22 (22)
Dredge 14 performing job 14 at time 0 - 62 (62)
Dredge 15 performing job 9 at time 0 - 77 (77)
Dredge 15 performing job 16 at time 151 - 182 (31)
Dredge 16 performing job 17 at time 0 - 18 (18)
Dredge 17 performing job 18 at time 0 - 57 (57)
Dredge 18 performing job 20 at time 334 - 343 (9)
Dredge 19 performing job 21 at time 0 - 2 (2)
Dredge 20 performing job 22 at time 0 - 4 (4)
Dredge 21 performing job 23 at time 0 - 5 (5)
Dredge 22 performing job 24 at time 0 - 78 (78)
Dredge 23 performing job 25 at time 273 - 310 (37)
Dredge 24 performing job 26 at time 0 - 20 (20)
Dredge 25 performing job 27 at time 0 - 3 (3)
Dredge 26 performing job 28 at time 60 - 7